In [1]:
import pandas as pd

In [6]:
railroad_lines = pd.read_csv("out.csv")
#for index, row in railroad_lines.iterrows():
#    print(row["internmediate"])
#    break

C:\Users\Jitendra Savanur\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (11,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
ND_railroad_lines = railroad_lines[railroad_lines["STATE"] == "ND"]
ND_railroad_lines.to_csv("ND_railroad_lines.csv", index = False)

In [9]:
len(ND_railroad_lines)
ND_railroad_lines_start_end = ND_railroad_lines[["start","end","MILES"]]
print(ND_railroad_lines_start_end.count())
#len(ND_railroad_lines_0_miles)
#ND_railroad_lines_start_end = ND_railroad_lines_start_end.drop_duplicates(subset = ["start","end"])
#ND_railroad_lines_start_end.MILES.fillna(0.001, inplace=True)
print(ND_railroad_lines_start_end.count())

start    2618
end      2618
MILES    2618
dtype: int64
start    2618
end      2618
MILES    2618
dtype: int64


In [10]:
import networkx as nx
#G=nx.Graph()
G=nx.MultiGraph()
for index, row in ND_railroad_lines.iterrows():
    G.add_edge(row["start"], row["end"], weight = float(row["MILES"]), key=row["OBJECTID"])

In [11]:
edges_list = G.edges()
count = 0
for edge in edges_list:
    count += 1
    #print(edge)
count

2618

In [43]:
src = "-100.745199039103539,46.802841590172207"
tgt = "-100.723349048749952,46.791685508090929"
p = nx.dijkstra_path(G, source=src, target=tgt)

In [44]:
len(p)

5

In [45]:
p

['-100.745199039103539,46.802841590172207',
 '-100.744070779361195,46.798858488827527',
 '-100.741630301076,46.797498736135097',
 '-100.74101331118942,46.797209506816998',
 '-100.723349048749952,46.791685508090929']

In [65]:
import folium
from folium.plugins import MarkerCluster
import math

ND_center = [47.5515, -101.0020]
my_map = folium.Map(location = ND_center, zoom_start = 6)

all_points = []
i = 0
while i < len(p) - 1:
    print(i)
    node = ND_railroad_lines[ND_railroad_lines["start"] == p[i]]
    node = node[node["end"] == p[i + 1]]
    node = node[node["MILES"] == node["MILES"].min()]
    flip = False
    if len(node) > 1:
        node = node.loc[[0]]
        #print("Multiple paths found for same node pair")
        #for index, row in node.iterrows():
         #   print(row["MILES"])
    elif len(node) == 0:
        print("Zero paths found for same node pair. Flipping start and end and checking again.")
        node = ND_railroad_lines[ND_railroad_lines["end"] == p[i]]
        node = node[node["start"] == p[i + 1]]
        node = node[node["MILES"] == node["MILES"].min()]
        flip = True
        #print(len(node))
    
    node.internmediate.fillna("")
        
    for index, row in node.iterrows():
        if flip == True:
            end = row["start"]
            start = row["end"]
        else:
            start = row["start"]
            end = row["end"]
        intermediates = row["internmediate"]

        #if math.isnan(intermediates):
        #    intermediate_coords = []
        #else:
        try:
            if ";" in intermediates:
                intermediate_coords = intermediates.split(";")
            else:
                intermediate_coords = [intermediates]
        except TypeError:
            intermediate_coords = []
            
        
        
        coords = []
        coords.append(float(start.split(",")[1]))
        coords.append(float(start.split(",")[0]))
        #folium.Marker(coords, popup = str(row["OBJECTID"])).add_to(my_map)
        all_points.append(coords)
        print("start:",start)
        
        print(intermediates)
        if flip == True:
            intermediate_coords.reverse()

        for intermediate_coord in intermediate_coords:
            coords = []
            coords.append(float(intermediate_coord.split(",")[1]))
            coords.append(float(intermediate_coord.split(",")[0]))
            #folium.Marker(coords, popup = str(row["OBJECTID"])).add_to(my_map)
            all_points.append(coords)
        
        coords = []
        coords.append(float(end.split(",")[1]))
        coords.append(float(end.split(",")[0]))
        #folium.Marker(coords, popup = str(row["OBJECTID"])).add_to(my_map)
        all_points.append(coords)
        print("end:",end)
    i += 1
    
coords = []
coords.append(float(src.split(",")[1]))
coords.append(float(src.split(",")[0]))
folium.Marker(coords, popup = "source").add_to(my_map)

coords = []
coords.append(float(tgt.split(",")[1]))
coords.append(float(tgt.split(",")[0]))
folium.Marker(coords, popup = "destination").add_to(my_map)

folium.PolyLine(all_points, color="red", weight=3.0, opacity=1).add_to(my_map)
#node0 = ND_railroad_lines[ND_railroad_lines["start"] == p[0]]
#node0 = node0[node0["end"] == p[1]]

0
start: -100.745199039103539,46.802841590172207
-100.745627340559651,46.801271147934052;-100.745693966807636,46.800938021954629;-100.74568444915721,46.800623932825538;-100.745636639021157,46.800314079998699;-100.745541570314629,46.800115063122917;-100.745484574904808,46.799995755215917;-100.745328171925621,46.799801815963846;-100.745246628254037,46.799700702285463;-100.744951574802343,46.799415167099014;-100.744589896899271,46.799158184935109
end: -100.744070779361195,46.798858488827527
1
start: -100.744070779361195,46.798858488827527
-100.743666666840753,46.798625185920976;-100.742771989733527,46.798120739790228
end: -100.741630301076,46.797498736135097
2
start: -100.741630301076,46.797498736135097
-100.741500462178095,46.797437870901064;-100.741382611297666,46.797382625800509;-100.741311958800566,46.797349506017611
end: -100.74101331118942,46.797209506816998
3
Zero paths found for same node pair. Flipping start and end and checking again.
start: -100.74101331118942,46.79720950681699

In [66]:
my_map